In [1]:
import pandas as pd
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt

In [2]:
remanga = pd.read_csv('./data/raw/remanga_catalog_full.csv')
mangalib = pd.read_csv('./data/raw/mangalib_catalog_titles.csv')
gmanga_matched = pd.read_csv('data/gmanga_matched_remanga_mangalib_ids.csv', sep=';')
gmint_matched = pd.read_csv('data/gmint_matched_remanga_mangalib_ids.csv', sep=';')

In [15]:
remanga.loc[:, 'id'] = list(range(len(remanga)))

In [9]:
gmanga_matched.head(2)

,Unnamed: 0,id,name,eng_name,original_name,another_names,chapters_count,remanga_id,mangalib_id,mangalib_id_1,n_duplicates_mangalib
0,0,1.0,Ван Пис,One Piece,One Piece,Budak Getah / Большой Куш / ワンピース / Один Кусок,1028.0,489.0,85.0,NaN,NaN
1,1,2.0,Наруто,Naruto,Naruto,ナルト,704.0,507.0,130.0,NaN,NaN


In [10]:
gmint_matched.head(2)

,id,name,eng_name,original_name,another_names,chapters_count,remanga_id,remanga_id_1,n_duplicates,mangalib_id,mangalib_id_1,n_duplicates_mangalib
0,11,Денег нет!,No money,Okane ga nai,お金がないっ / 午夜情深 / 没有钱,116,9766.0,NaN,NaN,NaN,NaN,NaN
1,18,Проклятие цвета крови,Crimson Spell,Crimson Spell,Kurimuzon Superu/クリムゾン・スペル/緋紅色的魔咒/緋色誘惑,75,10537.0,NaN,NaN,NaN,NaN,NaN


In [4]:
def get_matched(df):
    return df[(~df.remanga_id.isna() | (~df.mangalib_id.isna()))]

In [7]:
len(get_matched(gmint_matched) )/ len(gmint_matched)

0.03771236086701816

In [8]:
len(get_matched(gmanga_matched) )/ len(gmanga_matched)

0.5178622875252089

In [18]:
fetch_valid_ixs = lambda df, col: df[~df[col].isna()][col].tolist()
remanga_matched_ids = []
remanga_matched_ids += fetch_valid_ixs(gmanga_matched, 'remanga_id')
# remanga_matched_ids.append(gmanga_matched[~gmanga_matched.remanga_id_1.isna()].remanga_id_1.tolist())
remanga_matched_ids += fetch_valid_ixs(gmint_matched, 'remanga_id')
remanga_matched_ids += fetch_valid_ixs(gmint_matched, 'remanga_id_1')

In [20]:
remanga[~remanga.id.isin(remanga_matched_ids)]

,rus_name,en_name,issue_year,avg_rating,total_views,total_votes,type,dir,n_chapters,id
3,Герой иного мнения,Hero with another opinion,2020.0,9.2,458051,111061,Рукомикс,hero-with-another-opinion,9.0,3
7,"Архимаг, который вернулся спустя 4000 лет",The archmage who returned after 4000 years,2020.0,9.0,3786103,684219,Манхва,the-archmage-who-returned-after-4000-years,82.0,7
16,Воин-скелет не смог удержать подземелье,Skeleton Soldier Couldn't Protect the Dungeon,2018.0,9.2,3191592,533681,Манхва,skeleton_soldier_couldnt_protect_the_dungeon,141.0,16
28,Я убил Бессмертного,I Killed The Immortal,2021.0,9.1,74032,17948,Рукомикс,i-killed-the-immortal,3.0,28
29,"Раб ""Б"" ранга","""B"" rank slave",2020.0,7.9,1715203,290731,Манхва,b-rank-slave,58.0,29
...,...,...,...,...,...,...,...,...,...,...
16862,"Клочки, улитки и щенячьи хвостики",Snips and Snails and Puppy Dog Tails,2012.0,0.0,107,2,Манга,snips-and-snails-and-puppy-dog-tails,6.0,16862
16876,Козырь!,TRUMP!(JUNG Hyun-Joo),2008.0,0.0,605,0,Манхва,trumpjung-hyunjoo,16.0,16876
16880,Бронированные защитники справедливости эпохи Х...,Heisei Armored Justice Corps,1992.0,0.0,114,0,Манга,heisei-armored-justice-corps,6.0,16880
16881,Голубое семя,Blue Seed,1992.0,0.0,188,6,Манга,blue-seed,5.0,16881


In [33]:
remanga[~remanga.id.isin(remanga_matched_ids)].to_csv('./data/missing/remanga_exclusive.csv', sep=';')

In [31]:
mangalib_matched_ids = []
mangalib_matched_ids += fetch_valid_ixs(gmanga_matched, 'mangalib_id')
mangalib_matched_ids += fetch_valid_ixs(gmanga_matched, 'mangalib_id_1')
mangalib_matched_ids += fetch_valid_ixs(gmint_matched, 'mangalib_id')
mangalib_matched_ids += fetch_valid_ixs(gmint_matched, 'mangalib_id_1')

In [32]:
mangalib[~mangalib.id.isin(mangalib_matched_ids)]

,id,link,data_id,manga_type,name
2,2,poidyom-gulyat,67688,Руманга,Пойдём гулять!
8,8,bastard-hwang-youngchan,2035,Манхва,Сволочь
9,9,ranker-who-lives-a-second-time,12478,Манхва,"Ранкер, который живет второй раз"
11,11,nan-hao-shang-feng,18406,Маньхуа,Нань Хао и Шан Фэнь
23,23,ag-yeog-uiending-eunjug-eumppun,29916,Манхва,Единственный исход злодейки — смерть
...,...,...,...,...,...
1787,1787,great-sword,13710,Маньхуа,Легендарный бог меча
1788,1788,haikyuu-dj-ano-machi-ni-sunderu-karera-no-koto...,8833,Манга,"О тех, кто живет в том городе"
1793,1793,zhi-zun-kuang-di-xi-tong,35272,Маньхуа,Система Верховного Императора
1796,1796,my-fiance-is-in-love-with-my-little-sister,31862,Манга,Мой жених влюбился в мою младшую сестру


In [ ]:
mangalib[~mangalib.id.isin(mangalib_matched_ids)].to_csv('./data/missing/mangalib.csv', sep=';')